In [172]:
import pandas as pd
import numpy as np
import datetime as dt

Load Data

In [173]:
df = pd.read_parquet('data/yellow_tripdata_2018-01.parquet')

In [174]:
df1 = df.copy()  #Hacemos una copia para trabajar con los datos y para evitar que en un error afecte los datos originales

- VendorId sólo acepta 1 o 2, si se da algo fuera de esto se imputa por la moda.

In [175]:
supported_values = [1,2]    #creamos una variable con los valores admitidos
mode_vendor = int(df1['VendorID'].mode())   # la moda de la columna
#implementamos un bucle for para ir comprobando que cada valor esté dentro de los valores, caso contrario -
# se imputa por la moda
for i in df1['VendorID']:
    if i not in supported_values:
       df1['VendorID'].replace(to_replace= i,value= mode_vendor ,inplace=True) 

- tpep_pickup_datetime acepta registros entre 01-01-2018 y 31-01-2018 (23:59:59 hs), fuera de esto/NaT se imputa por el último valor dentro del rango. *Decisión final: Fuera del rango se desestima


In [176]:
min_date = dt.datetime(2018,1,1) #establecemos la fecha minima
max_date = dt.datetime(2018,2,1) # establecemos la fecha maxima
df1.query("tpep_pickup_datetime <= @max_date", inplace= True) #filtramos
df1.query("tpep_pickup_datetime >= @min_date", inplace= True)

- tpep_dropoff_datetime Acepta valores entre 01-01-2018 y 01-02-2018 (hasta el mediodía 12pm). Si se pasa del rango, imputar con el último valor dentro del rango.


In [177]:
min_date1 = dt.datetime(2018,1,1) #establecemos la fecha minima
max_date1 = dt.datetime(2018,2,1,12) # establecemos la fecha maxima (hasta el mediodia)

- tpep_dropoff_datetime se desestima, se reemplaza por el campo 'Duración del viaje'.

In [178]:
#Primero creamos la columna 'Travel_time'
df1['Travel_time'] = ((df1['tpep_dropoff_datetime'] - df1['tpep_pickup_datetime']).dt.seconds)/60

In [179]:
#Ahora procedemos a eliminar la columna 'tpep_dropoff_datetime'
df1 = df1.drop(['tpep_dropoff_datetime'],axis=1)

- Passanger_count acepta registros entre 1-4, fuera de esto se imputa por moda. *Decisión final: Imputar 0 por 1.


In [180]:
supported_values = [1,2,3,4]    #creamos una variable con los valores admitidos
mode_passenger = int(df1['passenger_count'].mode())   # la moda de la columna
#implementamos un bucle for para ir comprobando que cada valor esté dentro de los valores, caso contrario -
# se imputa por la moda (1).
for i in df1['passenger_count']:
    if i not in supported_values:
       df1['passenger_count'].replace(to_replace= i,value= mode_passenger ,inplace=True) 

- Trip_distance 3 sigmas para detectar outliers, se desestiman los outliers superiores. Los viajes con trip_distance = 0 se tratan de imputar usando el fare_amount + rate_code, si siguen sin poder ser imputados se desestiman.


In [181]:
below =  df1['trip_distance'].mean() - 3 * df1['trip_distance'].std()
above = df1['trip_distance'].mean() + 3 * df1['trip_distance'].std()
# filtramos los outliers superiores
df1.query("trip_distance < @above", inplace=True)

- PU_Location y DO_location aceptan valores dentro del rango (1, 265), fuera de este rango se imputa por 265 (Unknown). 


In [182]:
def imputeValue( p_x ):
    if (p_x>=1) and (p_x<=265):
        return p_x
    return 265

In [184]:
df1['PULocationID'] = df1['PULocationID'].apply(imputeValue)
df1['DOLocationID'] = df1['DOLocationID'].apply(imputeValue)

- Rate_CodeID acepta valores dentro del rango (1-6), fuera de este rango se imputa por moda.


In [185]:
mode = df1['RatecodeID'].mode()[0]

In [186]:
def imputeMode( p_x ):
    if p_x not in [1,2,3,4,5,6]:
        return mode
    return p_x

In [187]:
df1['RatecodeID'] = df1['RatecodeID'].apply(imputeMode)

- Store_n_fwd actualmente no está en el ERD, drop a la columna a menos de que se nos ocurra algún uso.*Decisión final: Drop

In [188]:
df1 = df1.drop(columns=['store_and_fwd_flag'])

- Payment_type acepta valores (1-6), fuera de este rango se imputa por 5 (Unknown).

In [189]:
def imputeType( p_x ):
    if p_x not in [1,2,3,4,5,6]:
        return 5
    return p_x

In [190]:
df1['payment_type'] = df1['payment_type'].apply(imputeType)

- Fare_Amount se trata de imputar los valores negativos usando trip_distance + rate_code, de lo contrario se desestiman (muchos de estos valores son payment_type 3-4, considerar marcarlos como outliers + desestimar).


In [ ]:
#Se agrega a la tabla de outliers, no se puede inferir los demás datos

- Extra acepta valores en (0.0, 0.5, 1.0) valores fuera del rango se imputan por moda.


In [192]:
mode_extra = df1['extra'].mode()[0]

In [193]:
def imputeExtra( p_x ):
    if p_x not in [0, 0.5, 1]:
        return mode_extra
    return p_x

In [194]:
df1['extra'] = df1['extra'].apply(imputeExtra)

- MTA_tax Sólo acepta valores de 0 y 0.5.*Decisión final: si el viaje no es outlier imputar

In [195]:
def imputeMTA( p_x ):
    if p_x != 0.5:
        return 0.5
    return p_x

In [196]:
df1['mta_tax'] = df1.loc[(~df1['payment_type'].isin([3,4])), 'mta_tax'].apply(imputeMTA)

- Improvement_surcharge Sólo acepta valores de (0.3), si el viaje no es outlier imputar. *Decisión final: imputar en todos los casos a 0.3


In [197]:
def imputeIS( p_x ):
    if p_x != 0.3:
        return 0.3
    return p_x

In [198]:
df1['improvement_surcharge'] = df1.loc[(~df1['payment_type'].isin([3,4])), 'improvement_surcharge'].apply(imputeIS)

- Tip_amount acepta cualquier valor, considerar dropear o no . * Decisión final: Dropear


In [199]:
df1 = df1.drop(['tip_amount'],axis=1)

#### Agregamos a un DataFrame de outliers los datos que no pueden ser tratados.

In [224]:
df1['Outlier'] = 0

In [226]:
df1.loc[ (df1['mta_tax'].isna())              , 'Outlier' ] = 1
df1.loc[ (df1['trip_distance']==0)            , 'Outlier' ] = 1
df1.loc[ (df1['fare_amount']<=0)              , 'Outlier' ] = 1
df1.loc[ (df1['improvement_surcharge'].isna()), 'Outlier' ] = 1
df1.loc[ ((df1['PULocationID']==265) & (df1['DOLocationID']==265)), 'Outlier' ] = 1

In [228]:
df_outliers = df1.loc[ df1['Outlier']==1 ]

In [229]:
df_outliers

,VendorID,tpep_pickup_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Travel_time,Outlier
58,2,2018-01-01 01:02:39,2,0.09,1,87,87,4,-3.0,0.0,NaN,0.00,NaN,-4.30,NaN,NaN,1.383333,1
63,1,2018-01-01 00:20:53,1,0.00,1,161,264,2,14.5,0.5,0.5,0.00,0.3,15.80,NaN,NaN,0.000000,1
114,1,2018-01-01 00:27:43,2,5.70,1,264,264,4,17.5,0.5,NaN,0.00,NaN,18.80,NaN,NaN,11.500000,1
127,1,2018-01-01 00:40:10,1,3.20,1,186,140,3,16.0,0.5,NaN,0.00,NaN,17.30,NaN,NaN,23.233333,1
162,1,2018-01-01 00:18:09,4,1.30,1,112,255,3,8.5,0.5,NaN,0.00,NaN,9.80,NaN,NaN,10.233333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8760376,2,2018-01-31 23:23:52,1,0.00,2,107,107,1,52.0,0.0,0.5,5.76,0.3,68.56,NaN,NaN,0.216667,1
8760436,2,2018-01-31 23:34:29,1,0.00,1,116,116,2,2.5,0.5,0.5,0.00,0.3,3.80,NaN,NaN,0.350000,1
8760551,2,2018-01-31 23:48:53,1,0.00,1,18,18,1,2.5,0.5,0.5,0.00,0.3,40.00,NaN,NaN,0.466667,1
8760685,2,2018-01-31 23:24:40,1,0.00,1,7,193,2,0.0,0.0,0.5,0.00,0.3,0.00,NaN,NaN,0.800000,1
